In [3]:
import pandas as pd
from pandas_gbq import read_gbq


# Define your BigQuery SQL query
query = """
    SELECT * FROM `web-scraping-2024.top_chart.top_chart` LIMIT 10
"""

# Load the data into a Pandas DataFrame
df = read_gbq(query, project_id='web-scraping-2024')
metadata = df['app_store_id'].apply(lambda x: f"https://apps.apple.com/US/app/id{x}")

Downloading: 100%|██████████|


In [27]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import re

# Function to fetch data from the app store page
def fetch_app_store_data(app_store_url):
    try:
        # Send a request to the app store URL
        response = requests.get(app_store_url, allow_redirects=True)

        # If redirection occurs, get the final URL
        scraping_url_redirect = response.url

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the required details
        title_with_rating = soup.find('h1', class_='product-header__title app-header__title').text.strip() if soup.find('h1', class_='product-header__title app-header__title') else None

        # Remove age rating (like "12+") from title
        title = re.sub(r'\s*\d+\+\s*$', '', title_with_rating) if title_with_rating else None

        subtitle = soup.find('h2', class_='product-header__subtitle app-header__subtitle').text.strip() if soup.find('h2', class_='product-header__subtitle app-header__subtitle') else None
        avg_rating = soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[0] if soup.find('figcaption', class_='we-rating-count star-rating__count') else None

        # Extract number of ratings and remove the word "Ratings"
        number_of_ratings = (
            soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[1].replace(' Ratings', '')
            if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        )

        developer_name = soup.find('dd', class_='information-list__item__definition').text.strip() if soup.find('dd', class_='information-list__item__definition') else None
        developer_url = soup.find('h2', class_='product-header__identity app-header__identity').find('a')['href'] if soup.find('h2', class_='product-header__identity app-header__identity') else None

        # Extract size from the specific tag by finding the 'Size' <dt>
        size = None
        size_term = soup.find('dt', string='Size')  # Use 'string' instead of 'text'
        if size_term:
            size_definition = size_term.find_next_sibling('dd')  # Get the corresponding <dd>
            size = size_definition.text.strip() if size_definition else None

        return {
            "scraping_url": app_store_url,
            "scraping_timestamp": datetime.now().isoformat(),
            "scraping_url_redirect": scraping_url_redirect,
            "title": title,
            "subtitle": subtitle,
            "avg_rating": float(avg_rating) if avg_rating else None,
            "number_of_ratings": number_of_ratings,  # Only the numeric part
            "developer_name": developer_name,
            "developer_url": developer_url,
            "size": size
        }
    except Exception as e:
        print(f"Error fetching data for {app_store_url}: {e}")
        return None

# Example usage:
# Assuming you have a DataFrame df with the app_store_url and other relevant columns.
scraped_data = []
for index, row in df.iterrows():
    app_store_url = f"https://apps.apple.com/us/app/id{row['app_store_id']}"  # Assuming you build the URL like this

    # Fetch app details
    app_data = fetch_app_store_data(app_store_url)

    if app_data:
        scraped_data.append(app_data)

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Create a DataFrame from the scraped data
df_scraped = pd.DataFrame(scraped_data)

# Ensure columns are in the specified order
df_scraped = df_scraped[[
    "scraping_url",
    "scraping_timestamp",
    "scraping_url_redirect",
    "title",
    "subtitle",
    "avg_rating",
    "number_of_ratings",
    "developer_name",
    "developer_url",
    "size"
]]


In [28]:
df_scraped.head()

,scraping_url,scraping_timestamp,scraping_url_redirect,title,subtitle,avg_rating,number_of_ratings,developer_name,developer_url,size
0,https://apps.apple.com/us/app/id6446901002,2024-09-24T16:28:08.137862,https://apps.apple.com/us/app/threads/id644690...,Threads,Share ideas & trends with text,4.6,508.4K,"Instagram, Inc.",https://apps.apple.com/us/developer/instagram-...,84.4 MB
1,https://apps.apple.com/us/app/id6448311069,2024-09-24T16:28:09.287213,https://apps.apple.com/us/app/chatgpt/id644831...,ChatGPT,The official app by OpenAI,4.9,1.1M,"OpenAI, L.L.C.",https://apps.apple.com/us/developer/openai/id1...,76.7 MB
2,https://apps.apple.com/us/app/id1641486558,2024-09-24T16:28:10.551317,https://apps.apple.com/us/app/temu-shop-like-a...,Temu: Shop Like a Billionaire,Shop incredible deals on Temu‪!‬,4.7,1.4M,Whaleco Inc.,https://apps.apple.com/us/developer/temu/id164...,69.6 MB
3,https://apps.apple.com/us/app/id284815942,2024-09-24T16:28:11.704740,https://apps.apple.com/us/app/google/id284815942,Google,"Search with AI, Images & Text",4.5,1.4M,Google LLC,https://apps.apple.com/us/developer/google/id2...,346.7 MB
4,https://apps.apple.com/us/app/id1542571008,2024-09-24T16:28:12.836307,https://apps.apple.com/us/app/gauth-ai-study-c...,Gauth: AI Study Companion,Ask anything and get answers,4.8,376K,GAUTHTECH PTE. LTD.,https://apps.apple.com/us/developer/gauthtech-...,92.7 MB


In [29]:
from google.cloud import storage
import pandas as pd
from datetime import datetime

# Function to upload DataFrame to GCS
def upload_dataframe_to_gcs(df, bucket_name):
    # Initialize a GCS client
    client = storage.Client()

    # Get the bucket
    bucket = client.bucket(bucket_name)

    # Create a filename with today's date
    today_date = datetime.now().strftime('%Y_%m_%d')  # Format: YYYY_MM_DD
    destination_blob_name = f'app_metadata_{today_date}.parquet'  # Example: app_data_2024_09_24.parquet

    # Convert the DataFrame to a Parquet file
    destination_file_name = f"/tmp/{destination_blob_name}"
    df.to_parquet(destination_file_name, index=False)

    # Create a blob object from the bucket
    blob = bucket.blob(destination_blob_name)

    # Upload the Parquet file to GCS
    blob.upload_from_filename(destination_file_name)

    print(f"Uploaded {destination_file_name} to gs://{bucket_name}/{destination_blob_name}")

# Example DataFrame to upload
# Assuming df_scraped is your DataFrame
upload_dataframe_to_gcs(df_scraped, 'app_metadata_top_chart')


Uploaded /tmp/app_metadata_2024_09_24.parquet to gs://app_metadata_top_chart/app_metadata_2024_09_24.parquet


In [30]:
from google.cloud import bigquery, storage

# Initialize clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

# Define your parameters
project_id = "web-scraping-2024"
dataset_id = "app_metadata"
bucket_name = "app_metadata_top_chart"
app_metadata_dir_prefix = ""
specific_file_name = "app_metadata_2024_09_24.parquet"

def list_parquet_files(bucket_name, prefix, specific_file):
    """List specific parquet file in a GCS bucket and prefix."""
    bucket = gcs_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.parquet') and specific_file in blob.name]

# Load the specific file
app_metadata_details_files = list_parquet_files(bucket_name, app_metadata_dir_prefix, specific_file_name)

if not app_metadata_details_files:
    print("No files found.")
else:
    app_metadata_table_ref = bq_client.dataset(dataset_id).table("app_metadata")
    app_metadata_job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,  # Parquet file format
        autodetect=True
    )

    # Start the load job for the specific file
    load_job = bq_client.load_table_from_uri(
        f"gs://{bucket_name}/{app_metadata_details_files[0]}",
        app_metadata_table_ref,
        job_config=app_metadata_job_config
    )

    # Wait for the job to complete
    load_job.result()

    # Check the result
    app_metadata_details_table = bq_client.get_table(app_metadata_table_ref)
    print(f"Loaded {app_metadata_details_table.num_rows} rows into {dataset_id}:app_metadata_top_chart.")


Loaded 10 rows into app_metadata:app_metadata_top_chart.
